In [2]:
import pandas as pd

In [3]:

df = pd.read_csv('../db/USDA_data.csv')

In [4]:
df.head()

,Symbol,Synonym Symbol,Scientific Name with Author,Common Name,Family
0,ABAB,NaN,Abutilon abutiloides (Jacq.) Garcke ex Hochr.,shrubby Indian mallow,Malvaceae
1,ABAB,ABAM5,Abutilon americanum (L.) Sweet,NaN,NaN
2,ABAB,ABJA,Abutilon jacquinii G. Don,NaN,NaN
3,ABAB,ABLI,Abutilon lignosum (Cav.) G. Don,NaN,NaN
4,ABAB70,NaN,Abietinella abietina (Hedw.) Fleisch.,abietinella moss,Thuidiaceae


In [5]:
df.shape

(93157, 5)

In [6]:
df['Scientific Name with Author'].value_counts()

Salix babylonica auct. non L.                                         2
Acer barbatum Michx. p.p.                                             2
Carex sonomensis Stacey                                               2
Paspalum platyphyllum Griseb., non Schult.                            2
Botrychium alaskense W.H. Wagner & J.R. Grant                         2
                                                                     ..
Trichophyllum tenue (Willd.) Farw.                                    1
Scirpus quadrangulatus Muhl., non Michx.                              1
Eleocharis capitata sensu S.F. Blake, non (L.) R. Br.                 1
Eleocharis tenuis (Willd.) Schult. var. tenuis                        1
Zygodon viridissimus (Dicks.) Brid. var. dentatus (Breidl.) Limpr.    1
Name: Scientific Name with Author, Length: 93150, dtype: int64

In [8]:
df['Scientific Name with Author'].describe()

count                             93157
unique                            93150
top       Salix babylonica auct. non L.
freq                                  2
Name: Scientific Name with Author, dtype: object

In [10]:
df['Common Name'].value_counts()

rim lichen                        127
dot lichen                        105
wart lichen                       104
lecidea lichen                    101
cup lichen                         92
                                 ... 
cutleaf burnweed                    1
San Clemente Island buckwheat       1
Santa Barbara Island buckwheat      1
whiteflower goldenbush              1
Reinwardt's zygodon moss            1
Name: Common Name, Length: 31718, dtype: int64

In [11]:
df['Common Name'].isna().sum()

49381

Some organisms (lichens mostly) have multiple scientific names associated to the same common name.

About half 50k entries don't have a common name asscoiated with it

## Extacting relevant data from db

In [12]:
# dropping na values in scientific name columnn
df_drop = df.dropna(subset=['Common Name'])
df_drop.shape

(43776, 5)

In [14]:
# Dropping sci names with duplicates.
df_drop.drop_duplicates(subset=['Scientific Name with Author'], keep='first', inplace=True)

C:\Users\sande\Documents\MobaXterm\slash\var\log\xwin\ipykernel_27680\3656619262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop.drop_duplicates(subset=['Scientific Name with Author'], keep='first', inplace=True)


In [15]:
df_drop.columns

Index(['Symbol', 'Synonym Symbol', 'Scientific Name with Author',
       'Common Name', 'Family'],
      dtype='object')

In [16]:
final = df_drop[['Symbol','Scientific Name with Author','Common Name']]

In [18]:
def std_name(name):
    word_list = name.split()
    #accession,region = word_list[0].split(':')

    sci_name = word_list[0]+' '+word_list[1]
    other = ' '.join(word_list[2:])
    #return accession,region,sci_name,other
    return sci_name,other

In [19]:
final[['scientific_name', 'other']] = final['Scientific Name with Author'].apply(lambda x: pd.Series(std_name(x)))

C:\Users\sande\Documents\MobaXterm\slash\var\log\xwin\ipykernel_27680\1522321515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final[['scientific_name', 'other']] = final['Scientific Name with Author'].apply(lambda x: pd.Series(std_name(x)))


In [20]:
final.head()

,Symbol,Scientific Name with Author,Common Name,scientific_name,other
0,ABAB,Abutilon abutiloides (Jacq.) Garcke ex Hochr.,shrubby Indian mallow,Abutilon abutiloides,(Jacq.) Garcke ex Hochr.
4,ABAB70,Abietinella abietina (Hedw.) Fleisch.,abietinella moss,Abietinella abietina,(Hedw.) Fleisch.
7,ABAL,Abronia alpina Brandegee,Ramshaw Meadows sand verbena,Abronia alpina,Brandegee
8,ABAL3,Abies alba Mill.,silver fir,Abies alba,Mill.
9,ABAM,Abies amabilis (Douglas ex Loudon) Douglas ex ...,Pacific silver fir,Abies amabilis,(Douglas ex Loudon) Douglas ex Forbes


In [21]:
final.to_csv('../db/common_name_USDA.csv', index=True)